In [40]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

In [36]:
data = pd.read_csv("/Users/mhausch/Data/Prod_Dashboards/Data/commercial_samples.csv")
data.shape, data.head()

((5601, 6),
   Date Received Forecast_Region     Time   Product        Type  Actuals
 0    2020-01-02              UK  Q1 2020  G360 CDx  Commercial        1
 1    2020-01-02              UK  Q1 2020  G360 CDx  Commercial        1
 2    2020-01-02          Turkey  Q1 2020  G360 CDx  Commercial        1
 3    2020-01-02          Turkey  Q1 2020  G360 CDx  Commercial        1
 4    2020-01-02           Spain  Q1 2020  G360 CDx  Commercial        1)

In [62]:
data.columns.to_list()

['Date Received', 'Forecast_Region', 'Time', 'Product', 'Type', 'Actuals']

In [61]:
conn = psycopg2.connect(
    host="database-2.cvu49ggyv9ee.eu-north-1.rds.amazonaws.com",
    port = 5432,
    user="postgres",
    password="postgres")

In [11]:
#conn.autocommit = True

In [52]:
cur = conn.cursor()

In [37]:
#Insert dataframe

def execute_many(conn, df, table):
    """
    Using cursor.executemany() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()

### SQAlchemy - Save pandas to Postgres

In [53]:
from sqlalchemy import URL
url_object = URL.create(
    "postgresql+psycopg2",
    username="postgres",
    password="postgres",  # plain (unescaped) text
    host="database-2.cvu49ggyv9ee.eu-north-1.rds.amazonaws.com",
    database="postgres",
)

In [54]:
engine = create_engine(url_object)

In [55]:
data.to_sql(
        'forecast', 
        con=engine, 
        index=False, 
        if_exists='replace')

601

### Save query in Postgres database to panda dataframe

In [60]:
# Function to query the postgres database and return the data as a dataframe
def sql_to_dataframe(conn, query, column_names):
   
   cursor = conn.cursor()
   try:
      cursor.execute(query)
   except (Exception, psycopg2.DatabaseError) as error:
      print("Error: %s" % error)
   cursor.close()
   return 1
   # The execute returns a list of tuples:
   tuples_list = cursor.fetchall()
   cursor.close()
   # Now we need to transform the list into a pandas DataFrame:
   df = pd.DataFrame(tuples_list, columns=column_names)
   return df

In [ ]:
conn = connect()
#loading our dataframe
df = sql_to_dataframe(conn, query, column_names)
#closing the connection
conn.close()
# Let’s see if we loaded the df successfully
df.head()

# Code to create table

In [39]:
execute_many(conn, data, "forecast")

Error: not all arguments converted during string formatting


1

In [28]:
command = (
        """
        CREATE TABLE forecast (
            forecast_id SERIAL PRIMARY KEY,
            product VARCHAR(255) NOT NULL,
            actuals INTEGER NOT NULL,
            forecast INTEGER NOT NULL,
            temp_forecast INTEGER NOT NULL,
            forecast_gap INTEGER NOT NULL
        )
        """)

In [29]:
cur.execute(command)

In [30]:
cur.close()

In [31]:
conn.commit()

In [32]:
conn.close()